In [3]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from typing import Tuple

In [4]:
data_dir = r"D:\riwa_v2"
images_dir = os.path.join(data_dir, r"images")
masks_dir = os.path.join(data_dir, r"masks")

train_data = []
for img, msk in zip(os.listdir(images_dir), os.listdir(masks_dir)):
   train_data.append({
       "image": os.path.join(images_dir, img),
       "annotation": os.path.join(masks_dir, msk)
   })

val_imgs = r"D:\riwa_v2\validation\images"
val_msks = r"D:\riwa_v2\validation\masks"
val_data = []
for img, msk in zip(os.listdir(val_imgs), os.listdir(val_msks)):
   val_data.append({
      "image": os.path.join(val_imgs, img),
      "annotation": os.path.join(val_msks, msk)
   })

test_dir = r"D:\riwa_v2\test"
t_img = os.path.join(test_dir, "images")
t_msk = os.path.join(test_dir, "masks")

test_data = []
for img, msk in zip(os.listdir(t_img), os.listdir(t_msk)):
   test_data.append({
       "image": os.path.join(t_img, img),
       "annotation": os.path.join(t_msk, msk)
   })


In [6]:
def load_image_and_mask(image_path: str, mask_path: str) -> Tuple[np.ndarray, np.ndarray]:
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if (mask is None) or (image is None):
        raise FileNotFoundError(f"file not found")
    
    return image, mask

def crop_image(
    dataset: dict,
    index: int,
    min_true_ratio: float,
    max_true_ratio: float,
    min_crop_size: Tuple[int, int],
    max_crop_size: Tuple[int, int],
    max_trial: int,
    save_path: str = r"D:\augmentation",
    show_result: bool = False
) -> Tuple[np.ndarray, np.ndarray]:
    """
    최소/최대 크기 범위와 참 값 비율 조건을 만족하도록 이미지를 자름.

    Parameters:
        image (np.ndarray): 원본 이미지 배열 (H, W, C).
        mask (np.ndarray): 마스크 배열 (H, W), 값은 0 또는 1.
        min_true_ratio (float): 포함해야 할 최소 참 값 비율 (0~1).
        min_crop_size (Tuple[int, int]): 최소 crop 크기 (min_height, min_width).
        max_crop_size (Tuple[int, int]): 최대 crop 크기 (max_height, max_width).

    Returns:
        cropped_image (np.ndarray): 조건을 만족하는 crop된 이미지 배열.
        cropped_mask (np.ndarray): 조건을 만족하는 crop된 마스크 배열.
    """
    img_name, msk_name = dataset[index].values()
    save_img = save_path + f"\\images\\{img_name.split('\\')[-1].split('.')[0]}_crop.jpg"
    save_msk = save_path + f"\\masks\\{msk_name.split('\\')[-1].split('.')[0]}_crop.png"
    image, mask = load_image_and_mask(img_name, msk_name)

    img_h, img_w = mask.shape
    min_h, min_w = min_crop_size
    max_h, max_w = max_crop_size

    if min_h > img_h or min_w > img_w: return
    
    if max_h > img_h: max_h = img_h
    if max_w > img_w: max_w = img_w
    
    if min_h > max_h or min_w > max_w: return
    
    for _ in range(max_trial):
        # 랜덤한 crop 크기 선택
        crop_h = np.random.randint(min_h, max_h + 1)
        crop_w = np.random.randint(min_w, max_w + 1)

        # 랜덤한 crop 시작 좌표 선택
        top = np.random.randint(0, img_h - crop_h + 1)
        left = np.random.randint(0, img_w - crop_w + 1)
        bottom = top + crop_h
        right = left + crop_w

        # 해당 영역의 마스크 추출
        cropped_mask = mask[top:bottom, left:right]

        mask_val = cropped_mask.astype(bool, copy=True)
        true_ratio = np.mean(mask_val)

        # 비율 조건 확인
        if true_ratio >= min_true_ratio and true_ratio <= max_true_ratio:
            cropped_image = image[top:bottom, left:right]
            cv2.imwrite(save_img, cropped_image)
            cv2.imwrite(save_msk, (cropped_mask).astype(np.uint8))
            if show_result:
                plt.subplot(121)
                plt.imshow(cropped_image)
                plt.subplot(122)
                plt.imshow(cropped_mask)
            return 

    # 조건을 만족하는 crop을 찾지 못한 경우
    print(f"failed to crop image {index}")

In [18]:
# 최소 및 최대 크기 범위와 최소 참 값 비율 설정
min_crop_size = (400, 400)  # 최소 크기
max_crop_size = (1024, 1024)  # 최대 크기
min_true_ratio = 0.3  # 최소 참 값 비율
max_true_ratio = 0.7  # 최대 참 값 비율
max_trial = 100

for i, _ in enumerate(train_data):
    crop_image(
        train_data, i, min_true_ratio, max_true_ratio, min_crop_size, max_crop_size, max_trial
    )

failed to crop image 3
failed to crop image 5
failed to crop image 14
failed to crop image 22
failed to crop image 26
failed to crop image 31
failed to crop image 37
failed to crop image 40
failed to crop image 47
failed to crop image 48
failed to crop image 49
failed to crop image 59
failed to crop image 61
failed to crop image 62
failed to crop image 63
failed to crop image 64
failed to crop image 67
failed to crop image 68
failed to crop image 70
failed to crop image 71
failed to crop image 72
failed to crop image 73
failed to crop image 74
failed to crop image 75
failed to crop image 85
failed to crop image 86
failed to crop image 89
failed to crop image 95
failed to crop image 96
failed to crop image 107
failed to crop image 108
failed to crop image 109
failed to crop image 110
failed to crop image 112
failed to crop image 113
failed to crop image 114
failed to crop image 115
failed to crop image 117
failed to crop image 119
failed to crop image 122
failed to crop image 124
failed

In [ ]:
# Horizontal Flip
save_path = r"D:\augmentation"

for i, d in enumerate(train_data):
    image_path, mask_path = d.values()
    image, mask = load_image_and_mask(image_path, mask_path)
    
    hflip_image = cv2.flip(image, 1)
    hflip_mask = cv2.flip(mask, 1)

    flip_img_path = save_path + "\\images\\" + image_path.split("\\")[-1].split(".")[0] + "_flip.jpg"
    flip_msk_path = save_path + "\\masks\\" + image_path.split("\\")[-1].split(".")[0] + "_flip.png"

    cv2.imwrite(flip_img_path, hflip_image)
    cv2.imwrite(flip_msk_path, hflip_mask)


In [23]:
import random

In [25]:
# Gaussian Blur
for i, d in enumerate(train_data):
    image_path, mask_path = d.values()
    image, mask = load_image_and_mask(image_path, mask_path)
    
    blur_image = cv2.GaussianBlur(image, (0, 0), random.uniform(2, 3))
    blur_mask = mask

    blur_img_path = save_path + "\\images\\" + image_path.split("\\")[-1].split(".")[0] + "_blur.jpg"
    blur_msk_path = save_path + "\\masks\\" + image_path.split("\\")[-1].split(".")[0] + "_blur.png"

    cv2.imwrite(blur_img_path, blur_image)
    cv2.imwrite(blur_msk_path, blur_mask)

In [ ]:
import albumentations as A

save_path = r"D:\augmentation"

# rainy noise
transform = A.Compose(
    [A.RandomRain(brightness_coefficient=0.9, drop_width=1, blur_value=5, p=1)],
)
for i, d in enumerate(train_data):
    image_path, mask_path = d.values()
    image, mask = load_image_and_mask(image_path, mask_path)

    rain_image = transform(image=image)
    rain_mask = mask

    plt.imshow(rain_image['image'])
    break
    # rain_img_path = save_path + "\\images\\" + image_path.split("\\")[-1].split(".")[0] + "_rain.jpg"
    # rain_msk_path = save_path + "\\masks\\" + image_path.split("\\")[-1].split(".")[0] + "_rain.png"

    # cv2.imwrite(rain_img_path, rain_image)
    # cv2.imwrite(rain_msk_path, rain_mask)

KeyboardInterrupt: 

In [ ]:
# foggy noise